# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [3]:
import os
import sys

sys.path.append('..')
# Set your preferred name
USER = "Yuxuan"
# Set the data and output directories
DATA_ROOT = '/home/yxcheng'
OUTPUT_ROOT = '../../output1'
TRAIT = "Huntington_s-Disease"

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [25]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [4]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

['TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Lower_Grade_Glioma_(LGG)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Prostate_Cancer_(PRAD)',
 'CrawlData.ipynb',
 'TCGA_Kidney_Papillary_Cell_Carcinoma_(KIRP)',
 'TCGA_Lung_Cancer_(LUNG)',
 'TCGA_Endometrioid_Cancer_(UCEC)',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)',
 'TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Glioblastoma_(GBM)',
 'TCGA_Melanoma_(SKCM)',
 'TCGA_Kidney_Clear_Cell_Carcinoma_(KIRC)',
 'TCGA_Mesothelioma_(MESO)',
 'TCGA_Rectal_Cancer_(READ)',
 'TCGA_Head_and_Neck_Cancer_(HNSC)',
 'TCGA_Large_Bcell_Lymphoma_(DLBC)',
 'TCGA_Uterine_Carcinosarcoma_(UCS)',
 'TCGA_Colon_Cancer_(COAD)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Bladder_Cancer_(BLCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)',
 'TCGA_Cervical_Ca

If no match is found, jump directly to GEO in Part 2.2

In [27]:
trait_subdir = "TCGA_Lung_Cancer_(LUNG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_LUNG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_HiSeqV2_PANCAN.gz')

In [28]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['ABSOLUTE_Ploidy',
 'ABSOLUTE_Purity',
 'AKT1',
 'ALK_translocation',
 'BRAF',
 'CBL',
 'CTNNB1',
 'Canonical_mut_in_KRAS_EGFR_ALK',
 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2',
 'EGFR']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_

In [ ]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'stopped_smoking_year', 'year_of_initial_pathologic_diagnosis', 'year_of_tobacco_smoking_onset']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [70.0, 67.0, 79.0, 68.0, 66.0],
 'days_to_birth': [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, nan, '121', nan, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': ['0', '1523', nan, '607', '426'],
 'days_to_new_tumor_event_after_initial_treatment': [nan,
  nan,
  nan,
  334.0,
  183.0],
 'stopped_smoking_year': [nan, nan, nan, nan, 1987.0],
 'year_of_initial_pathologic_diagnosis': [2009.0,
  2007.0,
  2007.0,
  2009.0,
  2009.0],
 'year_of_tobacco_smoking_onset': [nan, nan, nan, nan, 1963.0]}

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Lung-Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-05-4244-01,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.639586,-0.195527,-0.324695,0.193167,-0.572812
TCGA-05-4249-01,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.997486,-0.276027,-1.219395,1.850967,-0.471212
TCGA-05-4250-01,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.959514,0.714373,0.112305,-0.532833,-0.003812
TCGA-05-4382-01,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,1.013722,-1.587117,0.359267,2.59051,0.806623,0.812114,0.367373,-0.494595,-1.062233,0.108588
TCGA-05-4384-01,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,0.110214,1.469373,0.194605,1.404667,-0.290712


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1100 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lung-Cancer', the least common label is '0' with 110 occurrences. This represents 10.00% of the dataset.
The distribution of the feature 'Lung-Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 443 occurrences. This represents 40.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Yuxuan/Lung-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [5]:
dataset = 'GEO'
trait_subdir = "Huntington_s-Disease"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE135589', 'GSE147004', 'GSE154141', 'GSE147002', 'GSE147003']

Repeat the below steps for all the accession numbers

In [19]:
cohort = accession_num = "GSE147003"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/home/yxcheng/GEO/Huntington_s-Disease/GSE147003/GSE147003_family.soft.gz',
 '/home/yxcheng/GEO/Huntington_s-Disease/GSE147003/GSE147003_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [20]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"DNA methylation profiles from a transgenic sheep model of Huntington's disease"
!Series_summary	"An Infinium microarray platform (HorvathMammalMethylChip40) was used to generate DNA methylation data from n=168 blood samples of a transgenic sheep model of Huntington's disease."
!Series_summary	"84 transgenic sheep and age matched control sheep."
!Series_overall_design	"Two genotypes (transgenic) and control.  Roughly sex balanced (half females)."
!Series_overall_design	"Each sample characteristics value represents:"
!Series_overall_design	"characteristics: Basename = usual identifier of the methylation array comprised of the SENTRIX ID (i.e. chip identifier) and position (row and column)."
!Series_overall_design	"characteristics: Age (Years) = Age at the time of blood draw in units of years."
!Series_overall_design	"characteristics: Sex = female denotes female sheep."
!Series_overall_design	"characteristics: Genotype = Transgenic denotes Huntington disease status,"
!Serie

In [21]:
clinical_data.head()

,!Sample_geo_accession,GSM4412025,GSM4412026,GSM4412027,GSM4412028,GSM4412029,GSM4412030,GSM4412031,GSM4412032,GSM4412033,...,GSM4412183,GSM4412184,GSM4412185,GSM4412186,GSM4412187,GSM4412188,GSM4412189,GSM4412190,GSM4412191,GSM4412192
0,!Sample_characteristics_ch1,genotype huntington: Control,genotype huntington: Transgenic,genotype huntington: Control,genotype huntington: Transgenic,genotype huntington: Control,genotype huntington: Transgenic,genotype huntington: Control,genotype huntington: Transgenic,genotype huntington: Control,...,genotype huntington: Control,genotype huntington: Transgenic,genotype huntington: Control,genotype huntington: Transgenic,genotype huntington: Control,genotype huntington: Transgenic,genotype huntington: Control,genotype huntington: Transgenic,genotype huntington: Control,genotype huntington: Transgenic
1,!Sample_characteristics_ch1,age (years): 4.25,age (years): 4.25,age (years): 4,age (years): 4.25,age (years): 4.25,age (years): 2.916666667,age (years): 2.916666667,age (years): 3.666666667,age (years): 4.25,...,age (years): 4,age (years): 2.916666667,age (years): 4.25,age (years): 4.25,age (years): 4,age (years): 2.916666667,age (years): 4.25,age (years): 4.25,age (years): 5.227777778,age (years): 4.25
2,!Sample_characteristics_ch1,Sex: female,Sex: female,Sex: female,Sex: male,Sex: female,Sex: female,Sex: female,Sex: female,Sex: male,...,Sex: male,Sex: female,Sex: male,Sex: male,Sex: male,Sex: female,Sex: male,Sex: male,Sex: female,Sex: male
3,!Sample_characteristics_ch1,tissue: Blood,tissue: Blood,tissue: Blood,tissue: Blood,tissue: Blood,tissue: Blood,tissue: Blood,tissue: Blood,tissue: Blood,...,tissue: Blood,tissue: Blood,tissue: Blood,tissue: Blood,tissue: Blood,tissue: Blood,tissue: Blood,tissue: Blood,tissue: Blood,tissue: Blood
4,!Sample_characteristics_ch1,basename: 203203210131_R01C01,basename: 203203210131_R02C01,basename: 203203210131_R03C01,basename: 203203210131_R04C01,basename: 203203210131_R05C01,basename: 203203210131_R06C01,basename: 203203210131_R01C02,basename: 203203210131_R02C02,basename: 203203210131_R03C02,...,basename: 203203210187_R03C01,basename: 203203210187_R04C01,basename: 203203210187_R05C01,basename: 203203210187_R06C01,basename: 203203210187_R01C02,basename: 203203210187_R02C02,basename: 203203210187_R03C02,basename: 203203210187_R04C02,basename: 203203210187_R05C02,basename: 203203210187_R06C02


In [22]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['genotype huntington: Control', 'genotype huntington: Transgenic'],
 1: ['age (years): 4.25',
  'age (years): 4',
  'age (years): 2.916666667',
  'age (years): 3.666666667',
  'age (years): 5.758333333',
  'age (years): 5.238888889',
  'age (years): 3.863888889',
  'age (years): 6.436111111',
  'age (years): 5.75',
  'age (years): 5.741666667',
  'age (years): 6.986111111',
  'age (years): 5.997222222',
  'age (years): 3.891666667',
  'age (years): 5.747222222',
  'age (years): 5.244444444',
  'age (years): 6.997222222',
  'age (years): 3.9',
  'age (years): 5.227777778'],
 2: ['Sex: female', 'Sex: male'],
 3: ['tissue: Blood'],
 4: ['basename: 203203210131_R01C01',
  'basename: 203203210131_R02C01',
  'basename: 203203210131_R03C01',
  'basename: 203203210131_R04C01',
  'basename: 203203210131_R05C01',
  'basename: 203203210131_R06C01',
  'basename: 203203210131_R01C02',
  'basename: 203203210131_R02C02',
  'basename: 203203210131_R03C02',
  'basename: 203203210131_R04C02',
  'ba

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [33]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Psoriasis\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Psoriasis\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an intege

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [24]:
is_gene_availabe = True
trait_row = 0
age_row = 1
gender_row = 2

trait_type = 'binary'

In [25]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [26]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'genotype huntington: Transgenic':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age (years): '):
        try:
            return float(age_string.split('age (years): ')[1].rstrip(''))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format

def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'Sex: female':
        return 0
    elif gender_string == 'Sex: male':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'tissue: Hippocampus'
example_age = 'age: 23y'
example_gender = 'gender: Male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(0, None, None)

In [27]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)


,GSM4412025,GSM4412026,GSM4412027,GSM4412028,GSM4412029,GSM4412030,GSM4412031,GSM4412032,GSM4412033,GSM4412034,...,GSM4412183,GSM4412184,GSM4412185,GSM4412186,GSM4412187,GSM4412188,GSM4412189,GSM4412190,GSM4412191,GSM4412192
Huntington_s-Disease,0.00,1.00,0.0,1.00,0.00,1.000000,0.000000,1.000000,0.00,1.00,...,0.0,1.000000,0.00,1.00,0.0,1.000000,0.00,1.00,0.000000,1.00
Age,4.25,4.25,4.0,4.25,4.25,2.916667,2.916667,3.666667,4.25,4.25,...,4.0,2.916667,4.25,4.25,4.0,2.916667,4.25,4.25,5.227778,4.25
Gender,0.00,0.00,0.0,1.00,0.00,0.000000,0.000000,0.000000,1.00,0.00,...,1.0,0.000000,1.00,1.00,1.0,0.000000,1.00,1.00,0.000000,1.00


### Genetic data preprocessing and final filtering

In [28]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM4412025,GSM4412026,GSM4412027,GSM4412028,GSM4412029,GSM4412030,GSM4412031,GSM4412032,GSM4412033,GSM4412034,...,GSM4412183,GSM4412184,GSM4412185,GSM4412186,GSM4412187,GSM4412188,GSM4412189,GSM4412190,GSM4412191,GSM4412192
ID,,,,,,,,,,,,,,,,,,,,,
cg00000165,0.696791,0.645434,0.633041,0.610855,0.667496,0.582614,0.569999,0.596768,0.576174,0.691860,...,0.570973,0.559877,0.646140,0.538759,0.714180,0.673140,0.635783,0.645294,0.601172,0.584490
cg00001209,0.882646,0.899428,0.879341,0.885701,0.888487,0.882929,0.893199,0.873676,0.905101,0.892224,...,0.849176,0.905680,0.879754,0.881820,0.878871,0.907541,0.886740,0.877080,0.853736,0.860303
cg00001364,0.901995,0.910729,0.901996,0.884089,0.910420,0.920101,0.890174,0.907817,0.917927,0.906210,...,0.912268,0.923939,0.929560,0.922800,0.917359,0.931753,0.909672,0.922073,0.924660,0.921910
cg00001582,0.046551,0.040599,0.050694,0.045069,0.044785,0.042522,0.044694,0.041302,0.048841,0.043398,...,0.047361,0.050596,0.044477,0.047719,0.049545,0.044595,0.048683,0.048970,0.042278,0.044783
cg00002920,0.492209,0.513703,0.492087,0.502375,0.520136,0.497499,0.503536,0.500597,0.517282,0.504049,...,0.530951,0.513980,0.518154,0.491364,0.503261,0.514976,0.526438,0.495024,0.514352,0.521525


In [29]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['cg00000165',
 'cg00001209',
 'cg00001364',
 'cg00001582',
 'cg00002920',
 'cg00003994',
 'cg00004555',
 'cg00005112',
 'cg00005271',
 'cg00006213',
 'cg00008998',
 'cg00009248',
 'cg00009319',
 'cg00010445',
 'cg00010903',
 'cg00010926',
 'cg00010977',
 'cg00012521',
 'cg00012879',
 'cg00013189']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [30]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['cg00000165', 'cg00001209', 'cg00001364', 'cg00001582', 'cg00002920', 'cg00003994', 'cg00004555', 'cg00005112', 'cg00005271', 'cg00006213', 'cg00008998', 'cg00009248', 'cg00009319', 'cg00010445', 'cg00010903', 'cg00010926', 'cg00010977', 'cg00012521', 'cg00012879', 'cg00013189']\n"

If not required, jump directly to the gene normalization step

In [31]:
requires_gene_mapping = True

In [32]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['cg00000165', 'cg00001209', 'cg00001364', 'cg00001582', 'cg00002920'], 'Human.MAPINFO': ['91194674', '61220029', '214170376', '80828702', '130336474'], 'Human.CHR': ['chr1', 'chr5', 'chr1', 'chr10', 'chr10'], 'Human.Strand': ['R', 'F', 'R', 'R', 'R'], 'Design_Sequence': ['CATTTCAAAAGGGCAAAAGTCTTGGTCGTGAGCGTTCGGCCCCGGAGTCCATGCCACGGG[CG]ATGTGTGCCTCAGCTGTTCCATCAAAAGCCACTGTACTAACAGATCCTGACTGCACTTAG', 'TTTATCTTAGGAATTAATACATTCTGCCATTTTCTATTCTGGCTACAATCTAGAGAAATC[CG]TGAGGCTGAATACATTTTGTACCACTTTGTTCTGCTTATAATGTTAAGATGTTTTCAAGA', 'TGCCCCTTAATGCCACACTGGGGGAATGGCTCATACCCCGAATTATATTCTCAACCCGGG[CG]CGCTTTGCTCTCAGGTGCTCATCACATAAGCGATCCATATCAAACTGGCTCATAGTAGGC', 'CAGACAGTGAAAAGGAACTCGGCGTTAGGGGGTGAGTAAGTGAATCAACTCGCCCGGAGC[CG]AGCAAGTTGAAGTGTCAGATTACATTGGCTATTCTATTTCCAAAGGGCCCCGCGCCGCCG', 'AATGCAGCTTGCGCCAGCCACATCTGTGTCTTCCTCTTCTGCCCTCCAAATGAGCCACGC[CG]AGGCAGGGAAGCTGTTGACAAATTAATACTTTAAGAAAATAAACTATGGTCATGTTCAAC'], 'Forward_Sequence': ['CTAAGTGCAGTCAGGATCTGTTAGTACAGTGGCTTTTGATGGAACAGCTGAGGCACA

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [34]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['cg00000165', 'cg00001209', 'cg00001364', 'cg00001582', 'cg00002920'], 'Human.MAPINFO': ['91194674', '61220029', '214170376', '80828702', '130336474'], 'Human.CHR': ['chr1', 'chr5', 'chr1', 'chr10', 'chr10'], 'Human.Strand': ['R', 'F', 'R', 'R', 'R'], 'Design_Sequence': ['CATTTCAAAAGGGCAAAAGTCTTGGTCGTGAGCGTTCGGCCCCGGAGTCCATGCCACGGG[CG]ATGTGTGCCTCAGCTGTTCCATCAAAAGCCACTGTACTAACAGATCCTGACTGCACTTAG', 'TTTATCTTAGGAATTAATACATTCTGCCATTTTCTATTCTGGCTACAATCTAGAGAAATC[CG]TGAGGCTGAATACATTTTGTACCACTTTGTTCTGCTTATAATGTTAAGATGTTTTCAAGA', 'TGCCCCTTAATGCCACACTGGGGGAATGGCTCATACCCCGAATTATATTC

In [35]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'Human.Hg19_SYMBOL'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [36]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

8 input query terms found dup hits:	[('ABCA17P', 2), ('ALOX15P1', 2), ('APOC4-APOC2', 2), ('APOOP5', 2), ('ASB9P1', 2), ('CARMN', 2), ('
39 input query terms found no hit:	['1-Mar', 'ARNTL', 'BTBD11', 'C10orf99', 'C11orf44', 'C11orf49', 'C11orf74', 'C11orf95', 'C12orf10',
11 input query terms found dup hits:	[('CRIM1-DT', 3), ('CSNK1A1P1', 2), ('DENND10P1', 2), ('DLX6-AS1', 2), ('DOC2GP', 2), ('DPH6-DT', 2)
19 input query terms found no hit:	['CYHR1', 'DKFZP434H168', 'EEF1AKNMT', 'EFCAB1', 'ELMSAN1', 'FAM102B', 'FAM122B', 'FAM126A', 'FAM126
7 input query terms found dup hits:	[('FRG1BP', 2), ('GDNF-AS1', 2), ('GLRA4', 2), ('HOTTIP', 2), ('HOXD-AS2', 2), ('KLRC4-KLRK1', 2), (
9 input query terms found no hit:	['GATD3A', 'GBA', 'KIAA0100', 'KIAA0355', 'KIAA0556', 'KIAA1107', 'KIAA1109', 'KIAA1841', 'KIAA2026'
18 input query terms found dup hits:	[('LBX1-AS1', 2), ('LINC00229', 2), ('LINC00331', 2), ('LINC00332', 2), ('LINC00343', 2), ('LINC0046
35 input query terms found no hit:	['LINC00

In [37]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [38]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 168 samples.


In [39]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Huntington_s-Disease', the least common label is '0.0' with 84 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Huntington_s-Disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 3.8847222224999998
  50% (Median): 4.25
  75%: 4.25
Min: 2.916666667
Max: 6.997222222
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 73 occurrences. This represents 43.45% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [40]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

A new JSON file was created at: ../../output1/Yuxuan/Huntington_s-Disease/cohort_info.json


In [41]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [42]:
from utils import *

In [43]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'GSE147003'

In [44]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE147003,True,True,False,True,True,168,


In [45]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [46]:
merged_data.head()

,Huntington_s-Disease,Age,Gender,A1BG,A1CF,AACS,AADAT,AAK1,AATF,ABCA1,...,ZRANB2-AS1,ZSCAN29,ZSWIM3,ZSWIM5,ZSWIM6,ZSWIM8,ZXDC,ZYG11B,ZZEF1,ZZZ3
0,0.0,4.25,0.0,0.491806,0.899960,0.390774,0.463467,0.509208,0.917367,0.567127,...,0.602005,0.553851,0.651386,0.852842,0.804149,0.871325,0.166445,0.969788,0.940996,0.955394
1,1.0,4.25,0.0,0.490892,0.926637,0.408045,0.449019,0.538818,0.917649,0.566033,...,0.600293,0.561052,0.668281,0.852690,0.807688,0.878014,0.182718,0.972242,0.959748,0.956932
2,0.0,4.00,0.0,0.488184,0.886400,0.413423,0.461858,0.505386,0.913792,0.570111,...,0.598689,0.614170,0.663705,0.870609,0.806921,0.872751,0.152436,0.969765,0.939102,0.955801
3,1.0,4.25,1.0,0.499392,0.899555,0.390452,0.492872,0.513660,0.923118,0.572922,...,0.600632,0.336855,0.657022,0.869947,0.800576,0.871670,0.136181,0.971740,0.940295,0.951921
4,0.0,4.25,0.0,0.489025,0.903611,0.398884,0.456250,0.494501,0.927517,0.573857,...,0.593588,0.458641,0.658684,0.850950,0.806299,0.884765,0.147802,0.972008,0.949340,0.955066


In [47]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [48]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [49]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [50]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [51]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 51.52% ± 5.75%


In [52]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 62.42% ± 6.80%


In [53]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 5. Discussion and report

In [54]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0085
Found 125 genes with non-zero coefficients associated with the trait 'Huntington_s-Disease' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
 Variable  Coefficient  Absolute Coefficient
LINC01432    -5.859059              5.859059
     NEXN    -4.003604              4.003604
   TRIM39     3.943742              3.943742
    MSRB2    -3.918954              3.918954
  MIR5093     3.482877              3.482877
   TM7SF3    -3.211433              3.211433
    POLG2     3.176632              3.176632
    RIMS3    -2.977018              2.977018
LINC00210     2.432505              2.432505
  TSPAN15    -2.300655              2.300655
     ADAR    -2.224646              2.224646
     GOT1    -2.168314              2.168314
     PTH2    -2.162107              2.162107
     EDN1     2.155098              2.155098
     PRCC    -2.148877              2.148877

/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [55]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0085
Found 2 significant genes associated with the trait 'Huntington_s-Disease' conditional on the factor 'Age', with corrected p-value < 0.05:
Variable  Coefficient      p_value  corrected_p_value
     HTT     0.473079 6.638323e-79       4.479540e-75
    FLT1    -0.245933 5.910260e-12       1.994122e-08


/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
